In [59]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load data
edu_df = pd.read_excel("education_income.xls")
arrest_df = pd.read_csv("arrests.csv")

# edu metric
edu_df["EduMetric"] = (
    edu_df["Edu. Attainment: Bachelor's Degree (2010)"] +
    edu_df["Edu. Attainment: Postgraduate Degree (2010)"]
) / edu_df["Total Pop, 25 and older (2010)"]

# edu score
scaler = MinMaxScaler()
edu_df["Education Score (0-100)"] = scaler.fit_transform(edu_df[["EduMetric"]]) * 100

# arrest/safety score
arrest_counts = arrest_df.groupby("INCIDENTNEIGHBORHOOD").size().reset_index(name="Arrest_Count")
arrest_counts["Safety Score (0-100)"] = (1 - MinMaxScaler().fit_transform(arrest_counts[["Arrest_Count"]])) * 100

# merge datasets
combined = pd.merge(
    edu_df[["Neighborhood", "Education Score (0-100)"]],
    arrest_counts[["INCIDENTNEIGHBORHOOD", "Safety Score (0-100)"]],
    left_on="Neighborhood",
    right_on="INCIDENTNEIGHBORHOOD",
    how="inner"
)

In [47]:
# combining metrics
combined["Final Score (0-100)"] = (combined["Education Score (0-100)"] + combined["Safety Score (0-100)"]) / 2

In [48]:
# sort
df_sorted = combined.sort_values("Final Score (0-100)", ascending=False).reset_index(drop=True)

In [56]:
# parking "bestest" result
best_parking_neighborhood = "Whitehall"
print(f"Best parking accessibility: {best_parking_neighborhood} (Brownsville Rd)")


Best parking accessibility: Whitehall (Brownsville Rd)


In [57]:
# top 5
print("Top 5 neighborhoods:")
print(df_sorted[["Neighborhood", "Final Score (0-100)"]].head(5))

Top 5 neighborhoods:
     Neighborhood  Final Score (0-100)
0     South Shore            96.631365
1  Allegheny West            53.653408
2        West End            53.612380
3   Regent Square            51.182811
4  Chartiers City            50.678661


In [58]:
# BEST NEIGHBORHOOD
best = df_sorted.iloc[0]
print("Best neighborhood overall:", best["Neighborhood"])
print("Final score (0–100):", f"{best['Final Score (0-100)']:.2f}")
print("Best parking accessibility: Whitehall (Brownsville Rd)")

Best neighborhood overall: South Shore
Final score (0–100): 96.63
Best parking accessibility: Whitehall (Brownsville Rd)


# Conclusion
**The best neighborhood:**
## South Shore!!!

**The best neighborhood for parking:**
## Whitehall!!!

we had to implement the Parking Meters's metric differently due to that dataset's "location" column refering to roads instead of neighborhoods.